# How to Add a New Game

### Disclaimer
This notebook closely follows the tutorial steps from [docs/howto_add_games.md](https://github.com/clp-research/clembench/blob/main/docs/howto_add_games.md)
 in the [**clp-research/clembench**](https://github.com/clp-research/clembench) repository. <br> 
While the core content is derived from the original tutorial, some additional detailed descriptions and explanations have been added to serve as a source of knowledge during the development of our project.

## The GameBenchMark Class
The Benchmark is run for a particular game with **python scripts/cli.py run -g game -m model** <br>
- If only one model is defined for a two player game, it will represent both players.

## Execution
- When the run command is executed, the run routine in `benchmark.py` will determine the game code that needs to be invoked.
- The benchmark code loads all subclasses that inherit from GameBenchMark and calls `setup` on them.
- The setup method already loads the game instances **-->** every subclass is asked if it applies to the given name.
- That means, there has to be a subclass like the following for each game
- The subclass provides `GAME_NAME=taboo` and the rest is taken care of by the SuperClass
- Then the benchmark code checks if your game is single or multiplayer game (the default is multi-player).
- Then the run(dialog_pair,temperature) method is called by GameBenchmark.
- This is when the GameMaster becomes relevant (which is returned by thr `create_game_master()` factory method).

In [ ]:
class TabooGameBenchmark(GameBenchmark):

    def __init__(self):
        super().__init__(GAME_NAME)

    def get_description(self):
        return "Taboo game between two agents where one has to describe a word for the other to guess."

    def create_game_master(self, experiment: Dict, player_backends: List[str]) -> GameMaster:
        return Taboo(experiment, player_backends)
        
    def is_single_player(self) -> bool:
        return False

#### `get_description method`
This is returned when `python3 scripts/cli.py ls` is executed, see: <br>
```
2024-08-31 12:55:58,735 - benchmark.run - INFO - Listing benchmark games:
2024-08-31 12:55:58,736 - benchmark.run - INFO -  Game: cloudgame -> A simple game in which a player has to decide whether they see clouds or not.
2024-08-31 12:55:58,736 - benchmark.run - INFO -  Game: imagegame -> Image Game simulation to generate referring expressions and fill a grid accordingly
2024-08-31 12:55:58,736 - benchmark.run - INFO -  Game: matchit -> A simple game in which two players have to decide whether they see the same image or not.
2024-08-31 12:55:58,736 - benchmark.run - INFO -  Game: matchit -> A simple game in which two players have to decide whether they see the same image or not.
2024-08-31 12:55:58,736 - benchmark.run - INFO -  Game: matchit -> A simple game in which two players have to decide whether they see the same image or not.
2024-08-31 12:55:58,736 - benchmark.run - INFO -  Game: matchit -> A simple game in which two players have to decide whether they see the same image or not.
2024-08-31 12:55:58,737 - benchmark.run - INFO -  Game: privateshared -> Questioner and answerer in scorekeeping game.
2024-08-31 12:55:58,737 - benchmark.run - INFO -  Game: taboo -> Taboo game between two agents where one has to describe a word for the other to guess.
2024-08-31 12:55:58,737 - benchmark.run - INFO -  Game: textmapworld -> Graph Game.
2024-08-31 12:55:58,737 - benchmark.run - INFO -  Game: textmapworld_description -> Graph Game.
2024-08-31 12:55:58,737 - benchmark.run - INFO -  Game: textmapworld_graphreasoning -> Graph Game.
2024-08-31 12:55:58,737 - benchmark.run - INFO -  Game: textmapworld_questions -> Graph Game.
2024-08-31 12:55:58,738 - benchmark.run - INFO -  Game: textmapworld_specificroom -> Graph Game.
```

## GameMaster Class
- For each `experiment`in instances.json, that has been loaded with `on_setup()`, the game benchmark code applies the given dialog pair (or if not provided, tries to determine it based on instance information) <br>
- Each experiment represents a specific condition for the game, like difficulity and holds the actual game instance. <br>
- A `GameMaster` is created for each instance, by `self.create_gamemaster` method of the BenchMark. <br>
- The `GameMaster`is in charge of playing a single isntance of the game <br>
- For Taboo: Target word to be guessed, words that are not allowed to be said.

In [ ]:
try:
   game_master = self.create_game_master(experiment_config, dialogue_pair)
   # Recieve Game Information
   game_master.setup(**game_instance)
   # Coordinate actual Play
   game_master.play()
   # Stores the interactions between the players and GM in game_record_dir
   game_master.store_records(game_id, game_record_dir)
except Exception:  # continue with other episodes if something goes wrong
   self.logger.exception(f"{self.name}: Exception for episode {game_id} (but continue)")

## Relevant Classes
